In [ ]:
import torch
from torch import nn
from torchvision import transforms as tf
from models.mynn import functional as myf
from models.mynn.loss import ssim
from train import train
import typing
from torch.utils.data import Dataset

from skimage.data import shepp_logan_phantom
from skimage.transform import resize
import models as mymodels

from datasets import Fastmri_320p



In [2]:
# from torch import nn
# import torch
# import math
# from typing import List
# from einops import rearrange, repeat, pack, unpack

# class SinusoidalPositionEncoding1d(nn.Module):
#     """Sinusoidal Position Encoding for 1-dimensions"""
#     def __init__(self, d_model, pos_scale=1e2):
#         super(SinusoidalPositionEncoding1d, self).__init__()
#         self.d_model = d_model
#         self.pos_scale = pos_scale
#         self.conv1d = nn.Conv1d(1, d_model, 1)

#         self.register_buffer('_div_term', torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model)))

#     def forward(self, tensor = None, position = None):
#         """tensor:[batch_size, seq_len, d_model]  
#         position:[batch_size, seq_len, 1]  
#         tensor is None: return positional encoding  
#         position is None: return [0-1]*scale positional encoding for tensor  
#         """
#         assert tensor is not None or position is not None, "Either tensor or position must be provided"

#         if position is None:
#             position = torch.arange(tensor.size(1), device=tensor.device).unsqueeze(0).unsqueeze(-1).float()
#         position = (position - position.min()) / (position.max() - position.min())
#         position = position * self.pos_scale

#         assert len(position.size()) == 3, "position must be [batch_size, seq_len, 1]"

#         pe = torch.zeros(position.size(0), position.size(1), self.d_model, device=position.device)
#         pe[:, :, 0::2] = torch.sin(position * self._div_term)
#         pe[:, :, 1::2] = torch.cos(position * self._div_term)

#         if tensor is None:
#             return pe        
#         return tensor + pe




# class SinusoidalPositionEncodingmd(nn.Module):
#     """Multi-dimensional Sinusoidal Position Encoding"""
#     def __init__(self, d_model, pos_scale: float | List[float] = 1e2, n_dims=2):
#         super(SinusoidalPositionEncodingmd, self).__init__()
#         self.n_dims = n_dims
#         self.d_model = d_model
#         self.register_buffer('pos_scale', torch.tensor(pos_scale if isinstance(pos_scale, list) else [pos_scale] * n_dims))

#         assert d_model % n_dims == 0, "d_model must be divisible by n_dims"
#         self.register_buffer('_div_term', torch.exp(torch.arange(0, self.d_model // self.n_dims, 2).float() * -(math.log(10000.0) / self.d_model // self.n_dims)))

    
#     def forward(self, tensor, position = None):
#         """tensor:[batch_size, seq_len, d_model]  
#         position:[batch_size, seq_len, n_dims]  
#         tensor is None: return positional encoding   
#         position is None: return [0-1]*scale positional encoding for tensor   
#         """
#         assert tensor is not None or position is not None, "Either tensor or position must be provided"

#         if position is None:
#             position = []
#             for i in range(self.n_dims):
#                 pos = torch.arange(tensor.size(1), device=tensor.device).unsqueeze(0).unsqueeze(-1).float()
#                 position.append(pos)
#             position = torch.cat(position, dim=-1)
#         position = (position - position.min()) / (position.max() - position.min())
#         position = position * self.pos_scale

#         assert len(position.size()) == 3, "position must be [batch_size, seq_len, n_dims]"

#         pe = torch.zeros(position.size(0), position.size(1), self.d_model, device=position.device)
#         for i in range(self.n_dims): # [aaabbbccc]
#             pe[:, :, i * self.d_model // self.n_dims:(i+1) * self.d_model // self.n_dims:2] = torch.sin(position[:, :, i:i+1] * self._div_term)
#             pe[:, :, i * self.d_model // self.n_dims + 1:(i+1) * self.d_model // self.n_dims:2] = torch.cos(position[:, :, i:i+1] * self._div_term)

#         if tensor is None:
#             return pe
#         return tensor + pe


# class LearnablePositionEncoding(nn.Module):
#     """Learnable Position Encoding"""
#     def __init__(self, d_model, max_len=512):
#         super(LearnablePositionEncoding, self).__init__()
#         self.d_model = d_model
#         self.max_len = max_len
#         self._pe = nn.Parameter(torch.randn(max_len, d_model))

#     def forward(self, tensor):
#         return tensor + self._pe[:tensor.size(0), :]



# import torch
# from torch import nn

# from einops import rearrange, repeat, pack, unpack
# from einops.layers.torch import Rearrange

# class FeedForward(nn.Module):
#     def __init__(self, dim):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(dim, dim),
#             nn.GELU(),
#             nn.Linear(dim, dim),
#         )
#     def forward(self, x):
#         return self.net(x)


# class Attention(nn.Module):
#     def __init__(self, dim, heads = 8):
#         super().__init__()
#         inner_dim = dim *  heads
#         dim_head = dim // heads

#         self.heads = heads
#         self.scale = dim_head ** -0.5

#         self.attend = nn.Softmax(dim = -1)

#         self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)

#         self.to_out = nn.Sequential(
#             nn.Linear(inner_dim, dim),
#         )

#     def forward(self, x):
#         qkv = self.to_qkv(x).chunk(3, dim = -1)
#         q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)

#         dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

#         attn = self.attend(dots)

#         out = torch.matmul(attn, v)
#         out = rearrange(out, 'b h n d -> b n (h d)')
#         return self.to_out(out)

# class Transformer(nn.Module):
#     def __init__(self, dim, depth, heads):
#         super().__init__()
#         self.norm = nn.LayerNorm(dim, eps = 1e-13)
#         self.layers = nn.ModuleList([])
#         for _ in range(depth):
#             self.layers.append(nn.ModuleList([
#                 Attention(dim, heads = heads),
#                 FeedForward(dim),
#             ]))
#     def forward(self, x):
#         for attn, ff in self.layers:
#             x = attn(x) + x
#             x = x * 0.5
#             x = ff(x) + x
#             x = x * 0.5
#             x = self.norm(x)
#         return x
    
# class ViT(nn.Module):
#     def __init__(self, *, image_size, patch_size, dim, depth, heads, channels):
#         super().__init__()
#         image_height, image_width = image_size
#         patch_height, patch_width = patch_size

#         assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

#         num_patches = (image_height // patch_height) * (image_width // patch_width)

#         patch_dim = channels * patch_height * patch_width

#         dim = patch_dim

#         self.to_patch_embedding = nn.Sequential(
#             Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
#         )

#         self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))

#         self.transformer = Transformer(dim, depth, heads)

#         self.mlp_head = nn.Sequential(
#             nn.Linear(dim, dim),
#             nn.GELU(),
#             nn.Linear(dim, dim),
#             Rearrange('b (h w) (p1 p2 c) -> b c (h p1) (w p2)', h = image_height // patch_height, w = image_width // patch_width, p1 = patch_height, p2 = patch_width, c = channels),
#         )

#     def forward(self, series):
#         # b c h w 
#         x = self.to_patch_embedding(series)
#         b, n, _ = x.shape
#         x += self.pos_embedding[:, :(n)]

#         x = self.transformer(x)

#         recon = self.mlp_head(x)

#         return recon

In [3]:
# import models.mynn as mynn
# from models.mynn import functional as myf

# from torchkbnufft import KbNufft, KbNufftAdjoint, calc_tensor_spmatrix, calc_density_compensation_function, ToepNufft

# class MAE(nn.Module):
#     def __init__(self, 
#                  image_size = (320, 320),
#                  patch_size = (16,16),
#                  mask_ratio = 0.5,
#                  in_channels = 16, out_channels = 16,
#                 #  d_model_encoder = 1024, d_model_decoder = 1024,
#                  nlayers_encoder = 4, nlayers_decoder = 4,
#                  nheads_encoder = 8, nheads_decoder = 8,
#                  ):
#         super().__init__()

#         image_height, image_width = image_size
#         patch_height, patch_width = patch_size

#         assert image_height % patch_height == 0 and image_width % patch_width == 0, 'Image dimensions must be divisible by the patch size.'

#         num_patches = (image_height // patch_height) * (image_width // patch_width)
#         patch_dim = in_channels * patch_height * patch_width
#         d_model_encoder = patch_dim
#         d_model_decoder = patch_dim

#         self.image_size = image_size
#         # self.d_model_encoder = d_model_encoder
#         # self.d_model_decoder = d_model_decoder
#         self.nlayer_encoder = nlayers_encoder
#         self.nlayer_decoder = nlayers_decoder
#         self.nheads_encoder = nheads_encoder
#         self.nheads_decoder = nheads_decoder
#         self.mask_ratio = mask_ratio
#         self.in_channels = in_channels
#         self.out_channels = out_channels

#         ### Encoder
#         self.input_embed = nn.Sequential(
#             Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width),
#         )

#         self.norm_encoder = nn.LayerNorm(d_model_encoder)

#         self.pe_encoder = SinusoidalPositionEncoding1d(d_model = d_model_encoder, pos_scale=100)

#         self.encoder = Transformer(dim = d_model_encoder, depth = nlayers_encoder, heads = nheads_encoder)


#         ### Decoder
#         self.decoder_embed = nn.Linear(d_model_encoder, d_model_decoder, bias=True)

#         self.pe_decoder = SinusoidalPositionEncoding1d(d_model = d_model_decoder, pos_scale=100)

#         self.mask_token = nn.Parameter(torch.zeros(1, 1, d_model_decoder))

#         self.decoder = Transformer(dim = d_model_decoder, depth = nlayers_decoder, heads = nheads_decoder)

#         self.norm_decoder = nn.LayerNorm(d_model_decoder)

#         self.output_embed = nn.Sequential(
#             nn.Linear(d_model_decoder, d_model_decoder),
#             nn.GELU(),
#             nn.Linear(d_model_decoder, d_model_decoder),
#             Rearrange('b (h w) (p1 p2 c) -> b c (h p1) (w p2)', h = image_height // patch_height, w = image_width // patch_width, p1 = patch_height, p2 = patch_width, c = out_channels),
#         )


#     def mean_std_norm_complex(self, data: torch.Tensor, dim = [-1, -2, -3]):
#         real = data.real
#         imag = data.imag
#         real = (real - real.mean(dim=dim, keepdim = True)) / real.std(dim=dim, keepdim = True)
#         imag = (imag - imag.mean(dim=dim, keepdim = True)) / imag.std(dim=dim, keepdim = True)
#         return real + 1j * imag


#     def random_masking(self, x, mask_ratio):
#         """
#         https://github.com/facebookresearch/mae/tree/main

#         Perform per-sample random masking by per-sample shuffling.
#         Per-sample shuffling is done by argsort random noise.
#         x: [N, L, D], sequence
#         """
#         N, L, D = x.shape  # batch, length, dim
#         len_keep = int(L * (1 - mask_ratio))
        
#         noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
#         # sort noise for each sample
#         ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
#         ids_restore = torch.argsort(ids_shuffle, dim=1)

#         # keep the first subset
#         ids_keep = ids_shuffle[:, :len_keep]
#         x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

#         # generate the binary mask: 0 is keep, 1 is remove
#         mask = torch.ones([N, L], device=x.device)
#         mask[:, :len_keep] = 0
#         # unshuffle to get the binary mask
#         mask = torch.gather(mask, dim=1, index=ids_restore)

#         return x_masked, mask, ids_restore
    
#     def forward_encoder_train(self, x, mask_ratio):
#         # x: [batch, phase, readout * channel * 2]
#         # pos: [batch, phase, 1]

#         ### Encoder
#         # in training mode, the shape of kdata is matched with the shape of ktraj
#         # kdata should be masked as same as ktraj
#         x = self.pe_encoder(x)
#         x, mask, ids_restore = self.random_masking(x, mask_ratio)

#         # cls token
#         pass

#         # encode
        
#         x = self.encoder(x)
#         x = self.norm_encoder(x)

#         return x, mask, ids_restore
    
#     def forward_decoder_train(self, x, ids_restore):
#         # x: [batch, phase, readout * channel * 2]
#         # pos: [batch, phase, 1]

#         ### Decoder
#         # embed tokens
#         x = self.decoder_embed(x)

#         # append mask tokens to sequence
#         mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] - x.shape[1], 1)
#         x = torch.cat([x, mask_tokens], dim=1)
#         x = torch.gather(x, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle

#         # pe
#         x = self.pe_decoder(x)

#         # apply Transformer blocks
#         x = self.decoder(x)
#         x = self.norm_decoder(x)

#         # predictor projection
#         x = self.output_embed(x)

#         # remove cls token
#         pass

#         return x


#     def forward_train(self, x):
#         # seq: [batch, channel, height, weight]
#         x = self.input_embed(x) # [batch, seq, feature]
#         # encode
#         encoder_memory, mask, ids_restore = self.forward_encoder_train(x, self.mask_ratio)
        
#         # decode
#         x = self.forward_decoder_train(encoder_memory, ids_restore)

#         return {'pred':x, 'mask': mask}


#     def forward_encoder_eval(self, x):
#         # x: [batch, phase, readout * channel * 2]
#         # pos: [batch, phase, 1]

#         ### Encoder
#         # in evaluation mode, the shape of kdata is not matched with the shape of ktraj
#         # kdata should be masked as same as ktraj
#         x = self.pe_encoder(x)

#         # cls token
#         pass

#         # encode
#         x = self.encoder(x)
#         x = self.norm_encoder(x)

#         return x
    
#     def forward_decoder_eval(self, x):
#         # x: [batch, phase, readout * channel * 2]
#         # pos: [batch, phase, 1]

#         ### Decoder
#         # embed tokens
#         x = self.decoder_embed(x)

#         # pe
#         x = self.pe_decoder(x)

#         # apply Transformer blocks
#         x = self.decoder(x)
#         x = self.norm_decoder(x)

#         # predictor projection
#         x = self.output_embed(x)

#         # remove cls token
#         pass

#         return x
    
#     def forward_eval(self, x):
#         # kdata: [batch, channel, phase, readout]
#         # ktraj: [batch, phase, readout]
#         x = self.input_embed(x)
#         # encode
#         encoder_memory = self.forward_encoder_eval(x)

#         # decode
#         x = self.forward_decoder_eval(encoder_memory)

#         return x
    

#     def forward_loss_train(self, pred, gt):
#         pred, mask = pred['pred'], pred['mask']
#         pred = self.input_embed(pred)
#         gt = self.input_embed(gt)

#         loss = (pred - gt) ** 2
#         loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

#         loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
#         return loss
    
#     def forward_loss_eval(self, pred, gt):
#         loss = nn.functional.mse_loss(pred, gt)
#         return loss
    
#     def forward_loss(self, pred, gt):
#         if self.training:
#             return self.forward_loss_train(pred, gt)
#         else:
#             return self.forward_loss_eval(pred, gt)

#     # def forward_loss(self, imgs, pred, mask):
#     #     """
#     #     imgs: [N, 3, H, W]
#     #     pred: [N, L, p*p*3]
#     #     mask: [N, L], 0 is keep, 1 is remove, 
#     #     """
#     #     target = self.patchify(imgs)
#     #     if self.norm_pix_loss:
#     #         mean = target.mean(dim=-1, keepdim=True)
#     #         var = target.var(dim=-1, keepdim=True)
#     #         target = (target - mean) / (var + 1.e-6)**.5

#     #     loss = (pred - target) ** 2
#     #     loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

#     #     loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
#     #     return loss
    

#     def forward(self, x):
#         if self.training:
#             return self.forward_train(x)
#         else:
#             return self.forward_eval(x)



In [ ]:
# format: filename index_of_slice
trainlistpath = "/home/lyy/moco/datasets/.fastmri_320p/trainlist.txt"
vallistpath = "/home/lyy/moco/datasets/.fastmri_320p/vallist.txt"
testlistpath = "/home/lyy/moco/datasets/.fastmri_320p/testlist.txt"
showlistpath = "/home/lyy/moco/datasets/.fastmri_320p/showlist.txt"

with open(trainlistpath, 'r') as f:
    trainlist = f.readlines()
    trainlist = [(line.split()[0], int(line.split()[1])) for line in trainlist]

with open(vallistpath, 'r') as f:
    vallist = f.readlines()
    vallist = [(line.split()[0], int(line.split()[1])) for line in vallist]

with open(testlistpath, 'r') as f:
    testlist = f.readlines()
    testlist = [(line.split()[0], int(line.split()[1])) for line in testlist]

with open(showlistpath, 'r') as f:
    showlist = f.readlines()
    showlist = [(line.split()[0], int(line.split()[1])) for line in showlist]

print("file in trainlist: ", len(trainlist))
print("file in vallist: ", len(vallist))
print("file in testlist: ", len(testlist))
print("file in showlist: ", len(showlist))

In [5]:
# loadtransform
import torch
from torch import nn
from datasets import Fastmri_320p


load_keys = ['']

class Loadtransform(nn.Module):
    """Load the data to the memory"""
    def __init__(self):
        super().__init__()

    def forward(self, data):
        # input: ismrmrd_header rss csm kspace
        return {'pics_image':torch.tensor(data["pics_image"])}

loadtransform = Loadtransform()


In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# References:
# timm: https://github.com/rwightman/pytorch-image-models/tree/master/timm
# DeiT: https://github.com/facebookresearch/deit
# --------------------------------------------------------

from functools import partial

import torch
import torch.nn as nn

from timm.models.vision_transformer import PatchEmbed, Block

from utils.mae.util.pos_embed import get_2d_sincos_pos_embed


class MaskedAutoencoderViT(nn.Module):
    """ Masked Autoencoder with VisionTransformer backbone
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3,
                 embed_dim=1024, depth=24, num_heads=16,
                 decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
                 mlp_ratio=4., norm_layer=nn.LayerNorm, norm_pix_loss=False):
        super().__init__()
        self.in_chans = in_chans

        # --------------------------------------------------------------------------
        # MAE encoder specifics
        self.patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
        num_patches = self.patch_embed.num_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.blocks = nn.ModuleList([
            Block(embed_dim, num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(depth)])
        self.norm = norm_layer(embed_dim)
        # --------------------------------------------------------------------------

        # --------------------------------------------------------------------------
        # MAE decoder specifics
        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)

        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))

        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred =  nn.Sequential(
            nn.Linear(decoder_embed_dim, decoder_embed_dim),
            nn.GELU(),
            nn.Linear(decoder_embed_dim, patch_size**2 * in_chans, bias=True),
        ) # decoder to patch
        
        # --------------------------------------------------------------------------

        self.norm_pix_loss = norm_pix_loss

        self.initialize_weights()

    def initialize_weights(self):
        # initialization
        # initialize (and freeze) pos_embed by sin-cos embedding
        pos_embed = get_2d_sincos_pos_embed(self.pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token=True)
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        decoder_pos_embed = get_2d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        # initialize patch_embed like nn.Linear (instead of nn.Conv2d)
        w = self.patch_embed.proj.weight.data
        torch.nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        # timm's trunc_normal_(std=.02) is effectively normal_(std=0.02) as cutoff is too big (2.)
        torch.nn.init.normal_(self.cls_token, std=.02)
        torch.nn.init.normal_(self.mask_token, std=.02)

        # initialize nn.Linear and nn.LayerNorm
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            # we use xavier_uniform following official JAX ViT:
            torch.nn.init.xavier_uniform_(m.weight)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def patchify(self, imgs):
        """
        imgs: (N, self.in_chans, H, W)
        x: (N, L, patch_size**2 *self.in_chans)
        """
        p = self.patch_embed.patch_size[0]
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], self.in_chans, h, p, w, p))
        x = torch.einsum('nchpwq->nhwpqc', x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * self.in_chans))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size[0]
        h = w = int(x.shape[1]**.5)
        assert h * w == x.shape[1]
        
        x = x.reshape(shape=(x.shape[0], h, w, p, p, self.in_chans))
        x = torch.einsum('nhwpqc->nchpwq', x)
        imgs = x.reshape(shape=(x.shape[0], self.in_chans, h * p, h * p))
        return imgs

    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward_encoder_train(self, x, mask_ratio):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x, mask, ids_restore

    def forward_decoder_train(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x
    
    def forward_train(self, imgs, mask_ratio=0.75):
        latent, mask, ids_restore = self.forward_encoder_train(imgs, mask_ratio)
        pred = self.forward_decoder_train(latent, ids_restore)  # [N, L, p*p*3]
        return {'pred': pred,'mask': mask}
    
    def forward_encoder_eval(self, x, mask_ratio):
        # embed patches
        x = self.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)

        # append cls token
        cls_token = self.cls_token + self.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        return x, mask, ids_restore

    def forward_decoder_eval(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        x = self.unpatchify(x)

        return x
    
    def forward_eval(self, imgs):
        latent, _, ids = self.forward_encoder_eval(imgs, mask_ratio = 0.75)
        pred = self.forward_decoder_eval(latent, ids)  # [N, L, p*p*3]
        return pred

    def forward_loss_train(self, pred, gt):
        """
        imgs: [N, self.in_chans, H, W]
        pred: [N, L, p*p*self.in_chans]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        mask = pred['mask']
        pred = pred['pred']
        target = self.patchify(gt)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.e-6)**.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss
    
    def forward_loss_eval(self, imgs, pred):
        return nn.functional.mse_loss(imgs, pred)
    
    def forward_loss(self, pred, gt):
        if self.training:
            return self.forward_loss_train(pred, gt)
        else:
            return self.forward_loss_eval(pred, gt)
        
    def forward(self, x):
        if self.training:
            return self.forward_train(x)
        else:
            return self.forward_eval(x)
    



In [7]:
image_size = (320, 320)

model = MaskedAutoencoderViT(
    img_size = 320,
    patch_size = 16,
    in_chans = 1,
)


In [ ]:
# step1 mseloss
from datasets.phantom import Phantom
from models.mynn.loss.pytorch_msssim.ssim import SSIM


mse_fn = nn.MSELoss()

ssim_fn = SSIM(data_range=1, size_average=True, channel=1)

def loss_fn(x, y):
    mse_score = mse_fn(x, y)
    if mse_score < 0.01:
        return 1 - ssim_fn(x, y)
    return mse_score


# trainset = Fastmri_320p(trainlist, transform=loadtransform, disk_cache="20241225autoencoder")
# valset = Fastmri_320p(vallist, transform=loadtransform, disk_cache="20241225autoencoder")
# testset = Fastmri_320p(testlist, transform=loadtransform, disk_cache="20241225autoencoder")
# showset = Fastmri_320p(showlist, transform=loadtransform, disk_cache="20241225autoencoder")

# trainset.clean_disk_cache()
# valset.clean_disk_cache()
# testset.clean_disk_cache()
# showset.clean_disk_cache()

trainset = Phantom(image_size = (320,320))

from models.mynn import functional as myf

# def data_augmentation(data):
#     image = data['pics_image']
#     image = myf.complex_to_real(image)
#     image = (image - image.mean()) / image.std()
#     return {'pics_image':image}

# def input_transform(data):
#     return data['pics_image'], data['pics_image']

# def show_transform(pred, gt, batch = None):
#     if batch is None:
#         return myf.real_to_complex(pred).abs(), myf.real_to_complex(gt).abs()
#     return myf.real_to_complex(pred).abs(), myf.real_to_complex(gt).abs(),myf.real_to_complex(batch).abs()

def data_augmentation(data):
    image = data
    image = (image - image.mean()) / image.std()
    return {'pics_image':image}

def input_transform(data):
    return data['pics_image'], data['pics_image']

def show_transform(pred, gt, batch = None):
    if batch is None:
        return pred, gt
    return pred, gt, batch

train(model, trainset, model.forward_loss, epochs=300, batch_size=32, device='cuda:5',exp_name="phantom-mae", early_stopping_patience=50,early_stopping_after=20,
      epoch_length=640, eval_length_train = 64, eval_length_val = 64, input_transform=input_transform,augmentation_transform=data_augmentation,show_transform=show_transform,
      val_dataset=trainset,
      imshow_dataset=trainset)